In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [2]:
# 輸入訂房資訊
where_to_go = input('去哪裡旅遊?')
depart_time = input('出發日期?(請以yyyy-mm-dd輸入)')
back_time = input('回程日期?(請以yyyy-mm-dd輸入)')
man = int(input('幾個人一起去?'))

去哪裡旅遊?西雅圖
出發日期?(請以yyyy-mm-dd輸入)2020-04-13
回程日期?(請以yyyy-mm-dd輸入)2020-04-15
幾個人一起去?2


In [5]:
driver = webdriver.Chrome()
driver.get('https://www.booking.com')
time.sleep(2)
destination = driver.find_element_by_name('ss')
destination.send_keys(where_to_go)
checkin = driver.find_element_by_xpath('//*[@id="frm"]/div[1]/div[2]/div[1]/div[2]/div/div/div/div/span').click()
checkin = driver.find_element_by_xpath('//*[@id="frm"]/div[1]/div[2]/div[2]/div/div/div[3]')
checkin = driver.find_element_by_css_selector('td[data-date='+'"'+depart_time +'"'+']').click()
checkout = driver.find_element_by_css_selector('td[data-date='+'"'+back_time +'"'+']').click()
driver.find_element_by_xpath("//form[@id='frm']/div/div[4]/div[2]/button/span").click()

In [6]:
hotel_name_list = []
ranking_list = []
price_list = []
page_list = []
# 找出總共多少個頁面
current_link = driver.find_element_by_xpath('//*[@id="search_results_table"]/div[4]/nav/ul/li[2]/ul/li[1]/a')
url = current_link.get_attribute('href')
driver.get(url)
soup = BeautifulSoup(driver.page_source, 'html.parser')
for page in soup.find_all('div', {'class': 'bui-u-inline'}):
    page_list.append(page.text)
last_page = int(page_list[-1])

# 爬取所有頁面的訂房資料
for i in range(1,last_page):
    next_link = driver.find_element_by_xpath('//*[@id="search_results_table"]/div[4]/nav/ul/li[3]/a')
    url = next_link.get_attribute('href')
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(1)
    for ele in soup.find_all('div', {'class': 'sr_item_content sr_item_content_slider_wrapper '}):
        try:
            hotel_name = ele.find('span', {'class': 'sr-hotel__name'}).text
            ranking = ele.find('div', {'class': 'bui-review-score__badge'}).text
            price = ele.find('div', {'class': 'bui-price-display__value prco-inline-block-maker-helper'}).text
            hotel_name_list.append(hotel_name.split('\n')[1].split('（')[0])
            ranking_list.append(ranking)
            price_list.append(price.split('\n')[1])
        except AttributeError:
            pass
    time.sleep(1)

In [7]:
# 整理成DataFrame格式
choice = pd.DataFrame()
choice['hotel_name'] = hotel_name_list
choice['ranking'] = ranking_list
choice['price'] = price_list

In [8]:
choice.head(10)

,hotel_name,ranking,price
0,The Grove West Seattle Inn,8.5,"TWD 5,357"
1,Executive Inn by the Space Needle,8.1,"TWD 4,048"
2,Residence Inn by Marriott Seattle University D...,9.3,"TWD 7,162"
3,The Apple Theory Seattle,4.1,"TWD 3,491"
4,Stewart Place Apartments by Barsala,7.6,"TWD 4,186"
5,Staypineapple at Watertown Hotel,8.8,"TWD 7,111"
6,Pier 52 Residences by Barsala,7.8,"TWD 7,343"
7,Residence Inn by Marriott Seattle Downtown/Lak...,8.3,"TWD 11,664"
8,Westlake Luxe Apartments by Barsala,7.7,"TWD 6,741"
9,Mozart Guest House Seattle Bed and Breakfast,9.1,"TWD 5,056"
